In [2]:
import yaml
import wandb
from volume_dataloader import CTScanDataModule
from unet import UNet
from pytorch_lightning.loggers import WandbLogger
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import ModelCheckpoint
import torch

# wandb.login(relogin=True)

DATA_PATH = '/media/gaetano/DATA/DATA_NIFTI_JAWS/'

In [3]:
# Create YAML to paste into wandb sweep
sweep_config = {
    'method': 'random'
    # 'method': 'bayes'
}

metric = {
    'name': 'val_loss',
    'goal': 'minimize'
    # 'name': 'val_accu',
    # 'goal': 'maximize'
}

parameters_dict = {
    'loss_alpha': {
        'distribution': 'uniform',
        'min': 1e-2,
        'max': 1.0
    },
    'learning_rate': {
        'distribution': 'uniform',
        'min': 1e-5,
        'max': 1e-2
    },
    'start_filters': {
        'values': [8, 16, 32]
    },
    'n_blocks': {
        'values': [3, 4, 5, 6]
    },
    'loss_gamma': {
        'value': 1.0
    },
    'batch_size': {
        'value': 5
    },
    'in_channels': {
        'value': 1
    },
    'out_channels': {
        'value': 4
    },
    'dim': {
        'value': 3
    },
    'attention': {
        'value': False
    },
    'max_epochs': {
        'value': 50
    }
}

sweep_config['metric'] = metric
sweep_config['parameters'] = parameters_dict

with open('wandb_config.yaml', 'w') as file:
    documents = yaml.dump(sweep_config, file)

In [4]:
def train(config=None):
    with wandb.init(config=config): # project='utooth', entity='utooth'
        config = wandb.config

        data_loader = CTScanDataModule(DATA_PATH, batch_size=config.batch_size)
        model = UNet(in_channels=config.in_channels,
                     out_channels=config.out_channels,
                     n_blocks=config.n_blocks,
                     start_filters=config.start_filters,
                     activation='relu',
                     normalization='batch',
                     conv_mode='same',
                     dim=config.dim,
                     attention=config.attention,
                     loss_alpha=config.loss_alpha,
                     loss_gamma=config.loss_gamma,
                     learning_rate=config.learning_rate)
        checkpoint = ModelCheckpoint(monitor='val_loss')
        wandb.finish()
        wandb_logger = WandbLogger() # project='utooth', entity='utooth'

        trainer = Trainer(gpus=-1,
                          strategy='dp',
                          log_every_n_steps=1,
                          max_epochs=config.max_epochs,
                          auto_lr_find=False,
                          callbacks=[checkpoint],
                          logger=wandb_logger)
        torch.cuda.empty_cache()
        trainer.fit(model, data_loader)

In [5]:
# this creates a new sweep from the above config
# sweep_id = wandb.sweep(sweep_config, project='utooth', entity="utooth")

In [6]:
sweep_id_path = 'utooth/utooth/1bmrbrgq'
wandb.agent(sweep_id_path, train, count=5)

wandb: Agent Starting Run: xpggyzdb with config:
wandb: 	attention: False
wandb: 	batch_size: 5
wandb: 	dim: 3
wandb: 	in_channels: 1
wandb: 	learning_rate: 0.0027158550419628578
wandb: 	loss_alpha: 0.8759762676430869
wandb: 	loss_gamma: 1
wandb: 	max_epochs: 50
wandb: 	n_blocks: 3
wandb: 	out_channels: 4
wandb: 	start_filters: 32
wandb: Currently logged in as: playweird (use `wandb login --relogin` to force relogin)


GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


wandb: WARNING Config item 'in_channels' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'out_channels' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'n_blocks' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'start_filters' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'attention' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'dim' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'loss_alpha' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'loss_gamma' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'learning_rate' was locked by 'sweep' (ignored update).

  | Name       | Type       | Params
------------------------------------------
0 | down_convs | ModuleList | 859 K 
1 | up_convs   | ModuleList | 497 K 
2 | conv_final | Conv3d     | 132   
------------------------------------------
1.4 M     Trainable params
0         Non-train

Validation sanity check: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

...

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]